In [1]:
#import libraries
import pandas as pd
import numpy as np
import os
from pydataset import data

# acquire
from env import host, user, password

### acquire & summarize

1. Acquire data from mySQL using the python module to connect and query. You will want to end with a single dataframe. Make sure to include: the logerror, all fields related to the properties that are available. You will end up using all the tables in the database.
- Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid.
- Only include properties with a transaction in 2017, and include only the last transaction for each property (so no duplicate property ID's), along with zestimate error and date of transaction.
- Only include properties that include a latitude and longitude value.

2. Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

3. Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.

<hr style="border:3px solid black"> </hr>

#### #1 Acquire data from mySQL using the python module to connect and query. You will want to end with a single dataframe. Make sure to include: the logerror, all fields related to the properties that are available. You will end up using all the tables in the database.

In [2]:
def get_connection(db_name):
    '''
    This function uses my info from my env file to
    create a connection url to access the Codeup db.
    '''
    from env import host, user, password
    return f'mysql+pymysql://{user}:{password}@{host}/{db_name}'

In [3]:
def get_zillow():
    '''
    This function reads in the Zillow data from the Codeup db
    with properties_2017, predictions_2017 and propertylandusetype tables joined
    returns: a pandas DataFrame 
    '''
    
    zp_query = '''
    SELECT *
    FROM predictions_2017
    LEFT JOIN properties_2017 ON predictions_2017.parcelid = properties_2017.parcelid
    LEFT JOIN airconditioningtype ON properties_2017.airconditioningtypeid= airconditioningtype.airconditioningtypeid
    LEFT JOIN architecturalstyletype ON properties_2017.architecturalstyletypeid= architecturalstyletype.architecturalstyletypeid
    LEFT JOIN buildingclasstype ON properties_2017.buildingclasstypeid= buildingclasstype.buildingclasstypeid
    LEFT JOIN heatingorsystemtype ON properties_2017.heatingorsystemtypeid= heatingorsystemtype.heatingorsystemtypeid
    LEFT JOIN propertylandusetype ON properties_2017.propertylandusetypeid= propertylandusetype.propertylandusetypeid
    LEFT JOIN storytype ON properties_2017.storytypeid= storytype.storytypeid
    LEFT JOIN typeconstructiontype ON properties_2017.typeconstructiontypeid= typeconstructiontype.typeconstructiontypeid
    where properties_2017.latitude is not null and properties_2017.longitude is not null
    '''
    return pd.read_sql(zp_query, get_connection('zillow'))


In [4]:
df= get_zillow()

In [5]:
df.head()

,id,parcelid,logerror,transactiondate,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,...,buildingclasstypeid,buildingclassdesc,heatingorsystemtypeid,heatingorsystemdesc,propertylandusetypeid,propertylandusedesc,storytypeid,storydesc,typeconstructiontypeid,typeconstructiondesc
0,0,14297519,0.025595,2017-01-01,1727539,14297519,NaN,NaN,NaN,3.5,...,NaN,None,NaN,None,261,Single Family Residential,NaN,None,NaN,None
1,1,17052889,0.055619,2017-01-01,1387261,17052889,NaN,NaN,NaN,1.0,...,NaN,None,NaN,None,261,Single Family Residential,NaN,None,NaN,None
2,2,14186244,0.005383,2017-01-01,11677,14186244,NaN,NaN,NaN,2.0,...,NaN,None,NaN,None,261,Single Family Residential,NaN,None,NaN,None
3,3,12177905,-0.103410,2017-01-01,2288172,12177905,NaN,NaN,NaN,3.0,...,NaN,None,2.0,Central,261,Single Family Residential,NaN,None,NaN,None
4,4,10887214,0.006940,2017-01-01,1970746,10887214,1.0,NaN,NaN,3.0,...,NaN,None,2.0,Central,266,Condominium,NaN,None,NaN,None


In [6]:
df.shape

(77580, 77)

In [7]:
#shape: 77580, 77
#2932 rows with nulls in both logitude and latitude

In [8]:
df = df.sort_values('transactiondate').drop_duplicates('parcelid', keep= 'last')

In [9]:
df.shape

(77381, 77)

In [10]:
#new shape: ater dropping duplicates (77381, 77)
#199 rows with duplicates were dropped

<hr style="border:2px solid black"> </hr>

#### #2 Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

In [11]:
#summarize data in the df
#head, info, describe, value counts, nulls

def summarize(df):
    '''
    this function will take in a single argument (a pandas df) 
    output to console various statistics on said dataframe, including:
    #.head()
    #.info()
    #.describe()
    #.value_counts()
    #observation of nulls in the dataframe
    '''
    #print head
    print('=================================================')
    print('Dataframe head: ')
    print(df.head(3))
    
    #print info
    print('=================================================')
    print('Dataframe info: ')
    print(df.info())
    
    #print descriptive stats
    print('=================================================')
    print('Dataframe Description: ')
    print(df.describe())
    num_cols = [cols for col in df.columns if df[col].dtype != 'O']
    cat_cols = [cols for col in df.columns if col not in num_cols]
    
    #print value counts
    print('=================================================')
    print('Dataframe value counts: ')
    for col in df. columns:
        if col in cat_cols:
            print(df[col].value_counts())
        else:
            print(df[col].value_counts(bins=10, sort = False))
    
    #print nulls by column
    print('=================================================')
    print('nulls in dataframe by column: ')
    print(nulls_by_col(df))
    
    #print nulls by column
    print('=================================================')
    print('nulls in dataframe by row: ')
    print(nulls_by_row(df))
    print('=================================================')

In [12]:
summary_df = summarize(df)

Dataframe head: 
   id  parcelid  logerror transactiondate       id  parcelid  \
0   0  14297519  0.025595      2017-01-01  1727539  14297519   
1   1  17052889  0.055619      2017-01-01  1387261  17052889   
2   2  14186244  0.005383      2017-01-01    11677  14186244   

   airconditioningtypeid  architecturalstyletypeid  basementsqft  bathroomcnt  \
0                    NaN                       NaN           NaN          3.5   
1                    NaN                       NaN           NaN          1.0   
2                    NaN                       NaN           NaN          2.0   

   ...  buildingclasstypeid  buildingclassdesc  heatingorsystemtypeid  \
0  ...                  NaN               None                    NaN   
1  ...                  NaN               None                    NaN   
2  ...                  NaN               None                    NaN   

   heatingorsystemdesc  propertylandusetypeid        propertylandusedesc  \
0                 None          

AttributeError: 'DataFrame' object has no attribute 'dtype'

<hr style="border:2px solid black"> </hr>

#### #3 Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.

In [13]:
#get nulls by column
def nulls_by_col(df):
    num_missing = df.isnull().sum()
    rows = df.shape[0]
    prcnt_miss = num_missing / rows * 100
    cols_missing = pd.DataFrame({'num_rows_missing': num_missing, 'pct_rows_missing': prcnt_miss})
    return cols_missing

In [14]:
cols_missing = nulls_by_col(df)
cols_missing

,num_rows_missing,pct_rows_missing
id,0,0.000000
parcelid,0,0.000000
logerror,0,0.000000
transactiondate,0,0.000000
id,0,0.000000
parcelid,0,0.000000
airconditioningtypeid,52428,67.753066
architecturalstyletypeid,77175,99.733785
basementsqft,77331,99.935385
bathroomcnt,0,0.000000


<hr style="border:2px solid black"> </hr>

#### #4 Write a function that takes in a dataframe and returns a dataframe with 3 columns: the number of columns missing, percent of columns missing, and number of rows with n columns missing. Run the function and document takeaways from this on how you want to handle missing values.

In [15]:
#get nulls by row
def nulls_by_row(df):
    num_missing = df.isnull().sum(axis=1)
    prcnt_miss = num_missing / df.shape[1] * 100
    rows_missing = pd.DataFrame({'num_cols_missing': num_missing, 'pct_cols_missing': prcnt_miss})\
    .reset_index()\
    .groupby(['num_cols_missing', 'pct_cols_missing']).count()\
    .rename(index=str, columns={'index':'num_rows'}).reset_index()
    return rows_missing

In [16]:
rows_missing = nulls_by_row(df)
rows_missing

,num_cols_missing,pct_cols_missing,num_rows
0,25,32.467532467532465,2
1,26,33.76623376623377,11
2,27,35.064935064935064,21
3,28,36.36363636363637,39
4,29,37.66233766233766,43
5,30,38.961038961038966,36
6,31,40.25974025974026,230
7,32,41.55844155844156,178
8,33,42.857142857142854,4827
9,34,44.15584415584416,3304


<hr style="border:3px solid black"> </hr>

## Prepare:

#### #1. Remove any properties that are likely to be something other than single unit properties.

In [17]:
#this will show you the types
df.propertylandusedesc.value_counts()

Single Family Residential                     52320
Condominium                                   19294
Duplex (2 Units, Any Combination)              2009
Planned Unit Development                       1944
Quadruplex (4 Units, Any Combination)           727
Triplex (3 Units, Any Combination)              535
Cluster Home                                    333
Mobile Home                                      74
Manufactured, Modular, Prefabricated Homes       58
Residential General                              37
Cooperative                                      29
Commercial/Office/Residential Mixed Used         15
Townhouse                                         6
Name: propertylandusedesc, dtype: int64

In [ ]:
#remove all properties that landusetypeid does not qualify 
df = df[(df.propertylandusedesc == 'Single Family Residential') 
        (df.propertylandusedesc == 'Mobile Home') 
        (df.propertylandusedesc == 'Manufactured, Modular, Prefabricated Homes') 
        (df.propertylandusedesc == 'Townhouse')]


#### #2 Create a function that will drop rows or columns based on the percent of values that are missing: handle_missing_values(df, prop_required_column, prop_required_row).

In [18]:
def remove_outliers(df, k, col_list):
    for col in col_list:
        q1, q3 = df[f'{col}'].quartile([.25,.75]) #get quartiles
        iqr = q3-q1 #calculate iqr
        upper_bound = q3 + k * iqr
        upper_bound = q1 - k * iqr
        
        df = df[(df[f'{col}'] > lower_bound) & (df[f'{col}'] < upper_bound)]

#### #3 Decide how to handle the remaining missing values:
- Fill with constant value.
- Impute with mean, median, mode.
- Drop row/column

In [ ]:
#filling or imputing the data is not correct in this case
#it would slant the data too much
#removing nulls is the best choice for this case

In [ ]:
def handle_missing_values(df, prop_required_column = .60, prop_required_row = .60):
    threshold = int(round(prop_required_column*len(df.index),0))
    df.dropna(axis=1, thresh=threshold, inplace=True)
    threshold = int(round(prop_required_row*len(df.columns),0))
    df.dropna(axis=0, thresh=threshold, inplace=True)
    return df

<hr style="border:3px solid black"> </hr>

# Mall Data

1. Acquire data from mall_customers.customers in mysql database.
2. Summarize data (include distributions and descriptive statistics).
3. Detect outliers using IQR.
4. Split data (train, validate, and test split).
5. Encode categorical columns using a one hot encoder (pd.get_dummies).
6. Handles missing values.
7. Scaling

#### #1 Acquire data from mall_customers.customers in mysql database.

In [19]:
def get_mall_customers():
    '''
    This function reads in the mall_customers data from the Codeup db
    returns: a pandas DataFrame 
    '''
    
    zp_query = '''
    SELECT *
    FROM customers
    '''
    return pd.read_sql(zp_query, get_connection('mall_customers'))

In [20]:
df = get_mall_customers()

In [21]:
df.head()

,customer_id,gender,age,annual_income,spending_score
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40


#### #2 Summarize data (include distributions and descriptive statistics).

In [25]:
import wrangle

In [23]:
df = wrangle.summarize(df)

Dataframe head: 
   customer_id  gender  age  annual_income  spending_score
0            1    Male   19             15              39
1            2    Male   21             15              81
2            3  Female   20             16               6
Dataframe info: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   customer_id     200 non-null    int64 
 1   gender          200 non-null    object
 2   age             200 non-null    int64 
 3   annual_income   200 non-null    int64 
 4   spending_score  200 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 7.9+ KB
None
Dataframe Description: 
       customer_id         age  annual_income  spending_score
count   200.000000  200.000000     200.000000      200.000000
mean    100.500000   38.850000      60.560000       50.200000
std      57.879185   13.969007      26.264721       2

NameError: name 'cols' is not defined

#### #3 Detect outliers using IQR.

In [26]:
#age
age_outliers = wrangle.outlier_bound_calculation(df, 'age')

NameError: name 'np' is not defined

In [ ]:
#annual income
#spending score

#### #4 Split data (train, validate, and test split).

In [ ]:
#use train validate split

#### #5 Encode categorical columns using a one hot encoder (pd.get_dummies).

#### #6 Handles missing values

In [ ]:
#there are no missing values

#### #7 Scaling

In [ ]:
#use min_max_scaler
#age, income, spending score